In [1]:
from methods import load_data, select_route_data, group_nodes, group_links, obtain_routes, export_results
import pathlib

In [2]:
file = pathlib.Path.cwd() / "Enfield Saturn Routes AM test.xlsx"

In [3]:
#Load raw data
strategic_raw_data, qgis_table = load_data(file, file)

In [4]:
nodes = select_route_data(strategic_raw_data)
ogv_nodes = select_route_data(strategic_raw_data, ogv=True) #Extract routes for user class 9 (OGVs)

In [5]:
nodes_grouped=group_nodes(nodes)
ogv_nodes_grouped=group_nodes(ogv_nodes)

In [6]:
links = group_links(nodes_grouped)
ogv_links = group_links(ogv_nodes_grouped)

In [7]:
routes = obtain_routes(links, qgis_table)
ogv_routes=obtain_routes(ogv_links, qgis_table)

Separate gis sheet from same file

In [8]:
# Export results
export_results(routes, "uc_1-8_routes")
export_results(ogv_routes, "ogv_routes")

In [11]:
routes

[[204, 201, 172, 210, 205, 176, 194, 191, 181],
 [204, 202, 170],
 [204, 202, 169],
 [204, 201, 172, 210, 205, 176, 193, 206, 177, 199],
 [204, 201, 172, 210, 205, 176, 194, 191, 180, 195],
 [204, 201, 172, 210, 205, 176, 194, 191, 183],
 [208, 182, 189, 193, 206, 178, 167, 214, 209, 173, 203],
 [208, 182, 189, 193, 206, 178, 167, 214, 209, 172, 211, 219],
 [208, 182, 189, 193, 206, 178, 166, 217],
 [208, 182, 189, 193, 206, 178, 167, 214, 209, 173, 202, 170],
 [208, 182, 189, 193, 206, 178, 167, 214, 209, 173, 202, 169],
 [208, 182, 189, 193, 206, 177, 199],
 [208, 183],
 [208, 182, 190],
 [215, 172, 211, 220, 209, 173, 203],
 [215, 172, 210, 205, 176, 194, 191, 181],
 [215, 172, 211, 219],
 [215, 172, 210, 205, 176, 193, 206, 178, 166, 217],
 [215, 172, 211, 220, 209, 173, 202, 170],
 [215, 172, 211, 220, 209, 173, 202, 169],
 [215, 172, 210, 205, 176, 193, 206, 177, 199],
 [215, 172, 210, 205, 176, 194, 191, 180, 195],
 [215, 172, 210, 205, 176, 194, 191, 183],
 [215, 172, 210, 205,